모나드를 사용하면 모나드를 사용하지 않은 겨우 식을 다양한 순서로 평가할 수 있는 언어에서 평가 순서를 지정할 수 있다. 모나드를 사용해 a+b+c와 같은 식이 왼쪽에서 오른쪽 순서로 평가되게 만들 수 있다. 일반적으로는 이러한 모나드의 기능이 그리 유용해 보이지 않는다. 하지만 파일의 내용을 특정 순서대로 쓰고 읽고 싶은 경우, 모나드를 사용하면 read와 write 함수가 특정 순서대로 평가되도록 쉽게 만들 수 있다.

평가 순서가 다양하고, 최적화 컴파일러를 제공하는 언어의 경우 모나드를 사용하면 식을 평가하는 순서를 정할 수 있다는 장점이 있다. 파이썬의 경우 대부분의 부분이 엄격한 평가를 수행하고 최적화를 수행하지 않는다. 따라서 모나드를 실용적으로 활용할 수 있는 가능성을 별로 없다. 

그러나 PyMonad 모듈은 모나드 이상이다. 그 안에는 별도의 구현이 있는 함수형 프로그래밍 기능이 많이 들어 있다. PyMonad 모듈을 사용하면 표준 라이브러리에 있는 모듈만을 사용할 때 보다 더 간결하고 이해하기 쉬운 프로그램을 만들 수 있는 경우가 있다.

In [2]:
import pymonad

In [3]:
help(pymonad)

Help on package pymonad:

NAME
    pymonad

PACKAGE CONTENTS
    either
    io
    list
    maybe
    monad
    monoid
    operators (package)
    promise
    reader
    state
    tools
    writer

FILE
    c:\programdata\anaconda3\lib\site-packages\pymonad\__init__.py




### 함수적 합성과 커링

일부 함수형 언어는 인자가 많은 함수를 인자가 하나뿐인 여러 함수로 변환하는 방식으로 작동한다. 이러한 처리 과정을 커링이라 한다. 